In [26]:
OPENAI_API_KEY = ""

In [2]:
!pip install -q --upgrade langchain_community

In [3]:
from langchain_community.document_loaders import WebBaseLoader

# 3개의 블로그 포스팅 본문을 Load

In [4]:
loader = WebBaseLoader([
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ])
docs = loader.load()

# 불러온 본문을 Split (Chunking) : recursive text splitter 활용

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

splits = []
for doc in docs:
  splitted_docs = text_splitter.split_documents([doc])
  splits.extend(splitted_docs)

# Chunks 를 임베딩하여 Vector store 저장: openai 사용

In [6]:
!pip install langchain-openai

In [7]:
import os
from langchain_openai import OpenAIEmbeddings

# Set the API key
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
embeddings_model = OpenAIEmbeddings()

In [8]:
!pip install langchain-chroma sentence_transformers

In [9]:
# import
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    collection_name="sol_day2",
    documents=splits,
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

# User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve

In [10]:
query =  "agent momory"
searched_docs = vectorstore.similarity_search(query)

# print results
print(searched_docs)

[Document(page_content='Memory', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}), Document(page_content='memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g.', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and Ba

In [11]:
searched_docs[0]

Document(page_content='Memory', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"})

In [12]:
# save DB
db = Chroma.from_documents(docs, embeddings_model, persist_directory="./chroma_db")

# load DB
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings_model)


# User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성
- retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. (JsonOutputParser() 를 활용 ) - llama3 prompt format 준수

In [13]:
!pip install langchainhub

In [14]:
query =  "agent momory"

searched_docs = vectorstore.similarity_search(query)
contexts = [doc.page_content for doc in searched_docs]
print(contexts)

['Memory', 'memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g.', 'Memory can be defined as the processes used to acquire, store, retain, and later retrieve', 'Long-term memory: This provides the agent with the capability to retain and recall (infinite)']


In [15]:
from langchain.prompts import PromptTemplate

llama3_format_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

prompt = """
Check the <query> is relevant to retrieved <context>. Answer with "yes" or "no" in json format(key: relevance).

<query>
{query}
</query>

<context>
{context}
</context>
"""
prompt_llama3_format = llama3_format_template.format(prompt=prompt)

In [20]:
chain = prompt | llm
chain.invoke({"query": query, "context": context})

AIMessage(content='{\n  "relevance": "no"\n}', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1081fdf1-f750-4fee-8d52-57a26acc8871-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121})

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

query = "Memory"

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

vectorstore = Chroma.from_documents(
    collection_name="sol_day2",
    documents=splits,
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template(prompt_llama3_format)
model = ChatOpenAI(temperature=0)
output_parser = JsonOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
chain.invoke(query)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-cJLMa***************************************2Tf9. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# 5 에서 모든 docs에 대해 ‘no’ 라면 디버깅
(Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등)

# 5에서 ‘yes’ 라면 질문과 명확히 관련 없는 docs 나 질문
(예: ‘I like an apple’에 대해서는 ‘no’ 라고 나오는지 테스트 프롬프트 및 평가 코드 작성. 이 때는 관련 없다는 답변 작성)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

query = "I want to eat dinner."

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

vectorstore = Chroma.from_documents(
    collection_name="sol_day2",
    documents=splits,
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template(prompt_llama3_format)
model = ChatOpenAI(temperature=0)
output_parser = JsonOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
chain.invoke(query)

{'relevance': 'no'}